In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=ed12a9331664b465faff311c6fee8a0835ae5e4f5e8866342f9b38e3b93bf665
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

#Câu 1: Giỏ hàng

In [ ]:
spark = SparkSession.builder.appName("Baskets").getOrCreate()

In [ ]:
df = spark.read.csv("data.csv", header=True, inferSchema=True)
df.show()

+-------------+----------+--------------------+----+-----+---+-----------+
|Member_number|      Date|     itemDescription|year|month|day|day_of_week|
+-------------+----------+--------------------+----+-----+---+-----------+
|         1249|01/01/2014|        citrus fruit|2014|    1|  1|          2|
|         1249|01/01/2014|              coffee|2014|    1|  1|          2|
|         1381|01/01/2014|                curd|2014|    1|  1|          2|
|         1381|01/01/2014|                soda|2014|    1|  1|          2|
|         1440|01/01/2014|    other vegetables|2014|    1|  1|          2|
|         1440|01/01/2014|              yogurt|2014|    1|  1|          2|
|         1659|01/01/2014| specialty chocolate|2014|    1|  1|          2|
|         1659|01/01/2014|   frozen vegetables|2014|    1|  1|          2|
|         1789|01/01/2014|      hamburger meat|2014|    1|  1|          2|
|         1789|01/01/2014|             candles|2014|    1|  1|          2|
|         1922|01/01/2014

In [ ]:
# Sắp xếp dữ liệu theo Member_number, Date và tổng hợp danh sách món hàng
agg_df = (
    df.groupby("Member_number", "Date")
    .agg(F.collect_set("itemDescription").alias("itemDescription"))
    .orderBy("Member_number", "Date")
)

In [ ]:
agg_df.show()

+-------------+----------+--------------------+
|Member_number|      Date|     itemDescription|
+-------------+----------+--------------------+
|         1000|24/06/2014|[pastry, whole mi...|
|         1001|02/07/2014|[whole milk, saus...|
|         1002|26/04/2014|[whole milk, butter]|
|         1003|19/01/2014|[frozen meals, de...|
|         1003|27/02/2014|        [rolls/buns]|
|         1004|02/04/2014|[whole milk, pip ...|
|         1004|05/01/2014|[cling film/bags,...|
|         1004|07/01/2014|[canned beer, fro...|
|         1005|23/01/2014|[margarine, whipp...|
|         1006|07/05/2014|[softener, bottle...|
|         1009|24/01/2014|[canned fish, coc...|
|         1010|03/05/2014|[pip fruit, UHT-m...|
|         1011|06/02/2014|[pastry, curd che...|
|         1011|15/02/2014|[citrus fruit, bo...|
|         1012|22/06/2014|[frozen vegetable...|
|         1013|15/07/2014|[mustard, other v...|
|         1013|17/01/2014|[bottled water, n...|
|         1013|20/02/2014| [whole milk, 

In [ ]:
# Chuyển đổi cột itemDescription từ một danh sách thành một chuỗi được phân tách bằng dấu ","
agg_df = agg_df.withColumn("itemDescription", F.concat_ws(",", "itemDescription"))

In [ ]:
agg_df.show()

+-------------+----------+--------------------+
|Member_number|      Date|     itemDescription|
+-------------+----------+--------------------+
|         1000|24/06/2014|pastry,whole milk...|
|         1001|02/07/2014|whole milk,sausag...|
|         1002|26/04/2014|   whole milk,butter|
|         1003|19/01/2014|frozen meals,dent...|
|         1003|27/02/2014|          rolls/buns|
|         1004|02/04/2014|whole milk,pip fr...|
|         1004|05/01/2014|cling film/bags,d...|
|         1004|07/01/2014|canned beer,froze...|
|         1005|23/01/2014|margarine,whipped...|
|         1006|07/05/2014|softener,bottled ...|
|         1009|24/01/2014|canned fish,cocoa...|
|         1010|03/05/2014|pip fruit,UHT-mil...|
|         1011|06/02/2014|  pastry,curd cheese|
|         1011|15/02/2014|citrus fruit,bott...|
|         1012|22/06/2014|frozen vegetables...|
|         1013|15/07/2014|mustard,other veg...|
|         1013|17/01/2014|bottled water,nap...|
|         1013|20/02/2014|    whole milk

In [ ]:
# Ghi kết quả vào tệp baskets.csv với dấu ";" làm dấu phân cách
agg_df.write.option("delimiter", ";").mode("overwrite").csv("baskets.csv", header=True)

In [ ]:
a = spark.read.option("delimiter", ";") \
        .csv('/content/baskets.csv/part-00000-7841ffbb-e9f1-4871-a99b-1f26bba65075-c000.csv',
        header=True)

In [ ]:
a.show()

+-------------+----------+--------------------+
|Member_number|      Date|     itemDescription|
+-------------+----------+--------------------+
|         1000|24/06/2014|pastry,whole milk...|
|         1001|02/07/2014|whole milk,sausag...|
|         1002|26/04/2014|   whole milk,butter|
|         1003|19/01/2014|frozen meals,dent...|
|         1003|27/02/2014|          rolls/buns|
|         1004|02/04/2014|whole milk,pip fr...|
|         1004|05/01/2014|cling film/bags,d...|
|         1004|07/01/2014|canned beer,froze...|
|         1005|23/01/2014|margarine,whipped...|
|         1006|07/05/2014|softener,bottled ...|
|         1009|24/01/2014|canned fish,cocoa...|
|         1010|03/05/2014|pip fruit,UHT-mil...|
|         1011|06/02/2014|  pastry,curd cheese|
|         1011|15/02/2014|citrus fruit,bott...|
|         1012|22/06/2014|frozen vegetables...|
|         1013|15/07/2014|mustard,other veg...|
|         1013|17/01/2014|bottled water,nap...|
|         1013|20/02/2014|    whole milk

In [ ]:
!cat /content/baskets.csv/part-00000-197a01a3-f144-401c-aec6-fc8f8a9353c3-c000.csv

Member_number,Date,itemDescription
1000,24/06/2014,"pastry,whole milk,salty snack"
1001,02/07/2014,"whole milk,sausage,rolls/buns"
1002,26/04/2014,"whole milk,butter"
1003,19/01/2014,"frozen meals,dental care"
1003,27/02/2014,rolls/buns
1004,02/04/2014,"whole milk,pip fruit,tropical fruit"
1004,05/01/2014,"cling film/bags,dish cleaner"
1004,07/01/2014,"canned beer,frozen fish"
1005,23/01/2014,"margarine,whipped/sour cream"
1006,07/05/2014,"softener,bottled water,rolls/buns"
1009,24/01/2014,"canned fish,cocoa drinks"
1010,03/05/2014,"pip fruit,UHT-milk,coffee"
1011,06/02/2014,"pastry,curd cheese"
1011,15/02/2014,"citrus fruit,bottled water,rolls/buns,candy"
1012,22/06/2014,"frozen vegetables,brown bread"
1013,15/07/2014,"mustard,other vegetables"
1013,17/01/2014,"bottled water,napkins,hard cheese"
1013,20/02/2014,"whole milk,candy"
1013,26/03/2014,"frozen vegetables,tropical fruit"
1013,28/03/2014,"mustard,whipped/sour cream"
1014,09/04/2014,"butter milk,hamburger meat"
1014,10/06/2014,

# Câu 2: Apriori

In [ ]:
class APriori:
    def __init__(self, file_path, s, c):
        self.spark = SparkSession.builder.appName("APriori").getOrCreate()
        self.data = self.spark.read.option("delimiter", ";").csv(file_path, header=True, inferSchema=True)
        self.s = s
        self.c = c
        self.item_counts = None

    def get_frequent_pairs(self, pairs):
        min_support_count = self.data.count() * self.s
        frequent_pair_counts = pairs.groupby("item1", "item2").count()
        frequent_pair_counts = frequent_pair_counts.filter(F.col('count') > min_support_count)
        return frequent_pair_counts

    def get_association_rules(self, frequent_pairs):
        rules = (frequent_pairs.alias('fp')
                 .join(self.item_counts.alias('ic1'), F.col('fp.item1') == F.col('ic1.item'))
                 .join(self.item_counts.alias('ic2'), F.col('fp.item2') == F.col('ic2.item'))
                 .select('fp.item1', 'fp.item2', 'fp.count',
                         F.col('ic1.count').alias('item1_count'),
                         F.col('ic2.count').alias('item2_count')))

        rules = rules.withColumn('confidence', F.col('count') / F.col('item1_count'))
        rules = rules.filter(F.col('confidence') > self.c)
        return rules.select('item1', 'item2', 'confidence')

    def run(self):
        items = self.data.groupby("Member_number", "Date").agg(F.collect_list("itemDescription").alias("items"))
        items = items.withColumn('items', F.split(F.col('items').getItem(0), ','))
        self.item_counts = items.select(F.explode('items').alias('item')).groupby('item').count()
        self.item_counts = self.item_counts.filter(F.col('count') > self.data.count() * self.s)

        exploded_items = items.select("Member_number", "Date", F.posexplode(F.col("items"))).withColumnRenamed("pos", "position").withColumnRenamed("col", "item")

        pairs = (exploded_items.alias("left")
                 .join(exploded_items.alias("right"),
                       (F.col("left.Member_number") == F.col("right.Member_number")) &
                       (F.col("left.Date") == F.col("right.Date")) &
                       (F.col("left.position") < F.col("right.position")))
                 .select(F.col("left.Member_number"), F.col("left.Date"), F.col("left.item").alias("item1"), F.col("right.item").alias("item2")))

        frequent_pairs = self.get_frequent_pairs(pairs)
        association_rules = self.get_association_rules(frequent_pairs)

        frequent_pairs.write.mode("overwrite").csv('apriori_frequent_pairs.csv', header=True)
        association_rules.write.mode("overwrite").csv('apriori_association_rules.csv', header=True)

    def stop(self):
        self.spark.stop()

In [ ]:
apriori = APriori('baskets.csv', 0.005, 0.07)
apriori.run()
apriori.stop()

# Câu 3: PCY

In [ ]:
class PCY:
    def __init__(self, file_path, s, c):
        self.spark = SparkSession.builder.appName("PCY").getOrCreate()
        self.data = self.spark.read.option("delimiter", ";").csv(file_path, header=True, inferSchema=True)
        self.s = s
        self.c = c
        self.item_counts = None
        self.buckets = 5000
        self.bucket_count = {}

    def get_frequent_pairs(self, pairs):
        min_support_count = self.data.count()*self.s
        # Lọc bởi các buckets phổ biến
        frequent_buckets = [bucket for bucket, count in self.bucket_count.items() \
                            if count > min_support_count]
        frequent_pairs = pairs.filter(F.col('bucket').isin(frequent_buckets))
        # Tính số lượng mỗi cặp
        frequent_pair_counts = frequent_pairs.groupby("item1", "item2").count()

        # Lọc theo số lượng thực sự của cặp
        frequent_pair_counts = frequent_pair_counts.filter(F.col('count') > min_support_count)

        return frequent_pair_counts

    def get_association_rules(self, frequent_pairs):
        # Sử dụng DataFrame frequent_pairs để tạo ra các quy tắc liên kết
        # A -> B nếu support của {A, B} lớn hơn 's' và confidence của A -> B lớn hơn 'c'

        # Join frequent pairs với single item counts
        rules = (frequent_pairs.alias('fp')
                 .join(self.item_counts.alias('ic1'), F.col('fp.item1') == F.col('ic1.item'))
                 .join(self.item_counts.alias('ic2'), F.col('fp.item2') == F.col('ic2.item'))
                 .select('fp.item1', 'fp.item2', 'fp.count', \
                 F.col('ic1.count').alias('item1_count'), \
                 F.col('ic2.count').alias('item2_count')))\

        # Tính toán confidence
        rules = rules.withColumn('confidence', F.col('count') / F.col('item1_count'))

        # Lọc theo ngưỡng confidence
        rules = rules.filter(F.col('confidence') > self.c)

        return rules.select('item1', 'item2', 'confidence')

    def run(self):
        # Lần 1
        items = self.data.groupby("Member_number", "Date").agg(F.collect_list("itemDescription").alias("items"))
        # Chuyển đổi chuỗi 'a,b,c' thành ['a','b','c']
        items = items.withColumn('items', F.split(F.col('items').getItem(0), ','))
        self.item_counts = items.select(F.explode('items').alias('item')).groupby('item').count()
        self.item_counts = self.item_counts.filter(F.col('count') > self.data.count() * self.s)
        # Sử dụng posexplode để tạo một cột vị trí và cột mục
        exploded_items = items.select("Member_number", "Date", F.posexplode(F.col("items"))).withColumnRenamed("pos", "position").withColumnRenamed("col", "item")

        # Tự join để tạo cặp mục
        pairs = (exploded_items.alias("left")
                .join(exploded_items.alias("right"),
                      (F.col("left.Member_number") == F.col("right.Member_number")) &
                      (F.col("left.Date") == F.col("right.Date")) &
                      (F.col("left.position") < F.col("right.position")))
                .select(F.col("left.Member_number"), F.col("left.Date"), F.col("left.item").alias("item1"), F.col("right.item").alias("item2")))
        self.buckets = pairs.select("item1", "item2").distinct().count()
        pairs_with_buckets = pairs.withColumn("bucket", F.expr(f"abs(hash(struct(item1, item2))) % {self.buckets}"))
        bucket_counts_df = pairs_with_buckets.groupby("bucket").count()

        # Convert the dataframe to a Python dictionary
        self.bucket_count = bucket_counts_df.rdd.collectAsMap()

        # Trích xuất các cặp phổ biến và quy tắc liên kết
        frequent_pairs = self.get_frequent_pairs(pairs_with_buckets)
        association_rules = self.get_association_rules(frequent_pairs)

        # Lưu kết quả vào các tệp CSV
        frequent_pairs.write.mode("overwrite").csv('pcy_frequent_pairs.csv', header=True)
        association_rules.write.mode("overwrite").csv('pcy_association_rules.csv', header=True)

    def stop(self):
        self.spark.stop()

In [ ]:
pcy = PCY("baskets.csv", 0.005, 0.07)
pcy.run()
pcy.stop()

+-------------+----------+--------------------+
|Member_number|      Date|               items|
+-------------+----------+--------------------+
|         1000|24/06/2014|[pastry,whole mil...|
|         1001|02/07/2014|[whole milk,sausa...|
|         1002|26/04/2014| [whole milk,butter]|
|         1003|19/01/2014|[frozen meals,den...|
|         1003|27/02/2014|        [rolls/buns]|
|         1004|02/04/2014|[whole milk,pip f...|
|         1004|05/01/2014|[cling film/bags,...|
|         1004|07/01/2014|[canned beer,froz...|
|         1005|23/01/2014|[margarine,whippe...|
|         1006|07/05/2014|[softener,bottled...|
|         1009|24/01/2014|[canned fish,coco...|
|         1010|03/05/2014|[pip fruit,UHT-mi...|
|         1011|06/02/2014|[pastry,curd cheese]|
|         1011|15/02/2014|[citrus fruit,bot...|
|         1012|22/06/2014|[frozen vegetable...|
|         1013|15/07/2014|[mustard,other ve...|
|         1013|17/01/2014|[bottled water,na...|
|         1013|20/02/2014|  [whole milk,

# Câu 4: FPGrowth

In [ ]:
from pyspark.ml.fpm import FPGrowth

class FPGrowthModel:
    def __init__(self, file_path, s, c):
        self.spark = SparkSession.builder.appName("FPGrowthModel").getOrCreate()
        self.data = self.spark.read.option("delimiter", ";").csv(file_path, header=True, inferSchema=True)
        self.s = s
        self.c = c

    def run(self):
        # Chuẩn bị dữ liệu cho FPGrowth
        items = self.data.groupby("Member_number", "Date").agg(F.collect_list("itemDescription").alias("items"))
        # Chuyển đổi chuỗi 'a,b,c' thành ['a','b','c']
        items = items.withColumn('items', F.split(F.col('items').getItem(0), ','))

        items = items.select('items')

        # Tạo và train model FPGrowth
        fp_growth = FPGrowth(itemsCol="items", minSupport=self.s, minConfidence=self.c)
        model = fp_growth.fit(items)

        # Trích xuất tập phổ biến
        frequent_items = model.freqItemsets
        # Convert 'items' column from array to string
        frequent_items = frequent_items.withColumn("items", F.concat_ws(",", F.col("items")))
        frequent_items.write.mode("overwrite").csv('fpgrowth_frequent_items.csv', header=True)

        # Trích xuất các association rules
        association_rules = model.associationRules
        association_rules = association_rules.withColumn("antecedent", F.concat_ws(",", F.col("antecedent")))
        association_rules = association_rules.withColumn("consequent", F.concat_ws(",", F.col("consequent")))
        association_rules.write.mode("overwrite").csv('fpgrowth_association_rules.csv', header=True)

    def stop(self):
        self.spark.stop()


In [ ]:
fpg = FPGrowthModel("baskets.csv", 0.005, 0.07)
fpg.run()
fpg.stop()